# ONS
## Provisional Long-Term International Migration estimates

Transform [published data from ONS](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/internationalmigration/datasets/migrationstatisticsquarterlyreportprovisionallongterminternationalmigrationltimestimates) into [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data).

Firstly, scrape the web page to fetch the metadata and file URL.

In [1]:
from databaker.framework import *
import pandas as pd
import requests
from pathlib import Path
from io import BytesIO
from cachecontrol import CacheControl
from cachecontrol.caches.file_cache import FileCache
from cachecontrol.heuristics import LastModified
from lxml import html
from urllib.parse import urlparse, urljoin
import re

session = CacheControl(requests.Session(),
                       cache=FileCache('.cache'),
                       heuristic=LastModified())

pageURL = 'https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/internationalmigration/datasets/migrationstatisticsquarterlyreportprovisionallongterminternationalmigrationltimestimates'
page = session.get('https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/internationalmigration/datasets/migrationstatisticsquarterlyreportprovisionallongterminternationalmigrationltimestimates')
tree = html.fromstring(page.content)
title = tree.xpath("//h1/text()")[0]
releaseDate = pd.to_datetime(tree.xpath("//span[text() = 'Release date: ']/parent::node()/text()")[1]).tz_localize('Europe/London').isoformat()
mailto = tree.xpath("//span[text() = 'Contact: ']/following-sibling::a[1]/@href")[0]
fileURL = urljoin(pageURL, tree.xpath("//a[@title = 'Download as xls']/@href")[0])

Now load the latest spreadsheet directly.

In [2]:
sourceFolder = Path('in')
sourceFolder.mkdir(exist_ok=True)

response = session.get(fileURL)
fileName = re.findall('filename="([^"]+)"', response.headers['content-disposition'])[0]
inputFile = sourceFolder / fileName
with open(inputFile, 'wb') as f:
    f.write(response.content)
    
sheet = loadxlstabs(inputFile)
savepreviewhtml(sheet[1])

Loading in/provisionalestimatesoflongterminternationalmigrationyeseptember2017.xls which has size 1053184 bytes
Table names: ['Contents and Notes', 'Table 1', 'Table 2', 'Table 3', 'Table 4', 'Chart 1 TS', 'Chart 1 Net', 'Chart 2a', 'Chart 2b', 'Chart 3']


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,Click box to display these data as a time series chart,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,Highlight significant changes over the last year?,,,,,,,,,,1,,,,,,,,,,,,,,,,,,,,
,,Click box to view the overall composition of migration by citizenship,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Table 1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Long-Term International Migration,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Rolling annual data for the United Kingdom, year ending September 2017",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,United Kingdom


In [3]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

Fill out metadata

In [4]:
modifiedDate = pd.to_datetime('now').tz_localize('Europe/London').isoformat()

from string import Template
with open(Path('metadata') / 'dataset.trig.template', 'r') as metadata_template_file:
    metadata_template = Template(metadata_template_file.read())
    with open(destinationFolder / 'dataset.trig', 'w') as metadata_file:
        metadata_file.write(metadata_template.substitute(
            title=title,
            releaseDate=releaseDate,
            mailto=mailto,
            modifiedDate=modifiedDate,
        ))